In [ ]:
import sys, os 
import pandas as pd 
import numpy as np
import utils

from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_recall_fscore_support, classification_report, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer


from sklearn.neural_network import MLPClassifier

In [ ]:
def load_data():

    data = os.path.join("data", "train.csv")

    df = pd.read_csv(data)
    X_train = df[['id', 'comment_text']]
    y_train = df[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

    X_test = pd.read_csv(os.path.join("data", "test.csv"))
    y_test = pd.read_csv(os.path.join("data", "test_labels.csv"))
    test = X_test.merge(y_test, on='id')
    test = test[ (test['toxic']!=-1) | (test['severe_toxic']!=-1) | 
                (test['obscene']!=-1) | (test['threat']!=-1) | (test['insult']!=-1) 
                | (test['identity_hate']!=-1) ]
    test = test.reset_index(drop=True) 
    
    X_test = test[['id', 'comment_text']]
    y_test = test[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']]

    return X_train, y_train, X_test, y_test 

In [ ]:
# Constants 

ngrams = 1
penalty = 'l2'
Cs = 10
cv = 5
max_iter = 1000 
scoring = 'accuracy'

In [ ]:
X_train, y_train, X_test, y_test = utils.load_data()
np.random.seed(42)

In [ ]:
y_train

In [ ]:
vectorizer = TfidfVectorizer(ngram_range=(1, ngrams))

In [ ]:
def tfidf_transform(corpus, is_train=False):
    if is_train: 
        X = vectorizer.fit_transform(corpus)
    else:
        X = vectorizer.transform(corpus)
    return X

In [ ]:
def print_results(model, X, y):
    """
    Evaluate the model, prints results, and return as tuple
    """
    prediction = model.predict(X)
    precision, recall, fbeta_score, support = \
        precision_recall_fscore_support(y, prediction)
    accuracy = accuracy_score(y, prediction)

    print ("Precision: {}\nRecall: {}\nF-Score: {}\nSupport: {}\nAccuracy {}\n".format(
            precision, recall, fbeta_score, support, accuracy))

    print (classification_report(y, prediction))

    return (precision, recall, fbeta_score, support, accuracy)

In [ ]:
# turn the corpus into a list to pass into the vectorizer 
train_corpus = X_train['comment_text'].values.tolist()
test_corpus = X_test['comment_text'].values.tolist()

X_train_tfidf = tfidf_transform(train_corpus, is_train=True)
X_test_tfidf = tfidf_transform(test_corpus, is_train=False)

In [ ]:
def run_model(): 
    labels = ['toxic' , 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    
    for label in labels: 
        print("Train model on the \"{}\" label.\n".format(label))
        y_train_label = y_train[label].values.tolist()
        y_test_label = y_test[label].values.tolist()
        
        lr = LogisticRegressionCV(   class_weight="balanced", 
                                     Cs = Cs, 
                                     cv = cv, 
                                     penalty = penalty, 
                                     scoring = scoring, 
                                     max_iter = max_iter)
        
        lr.fit(X_train_tfidf, y_train_label)
        
        print("Train Results\n")
        print_results(lr, X_train_tfidf, y_train_label)
        
        print("Test Results\n")
        
        print_results(lr, X_test_tfidf, y_test_label)

In [ ]:
run_model()



In [ ]:
X_train, y_train, X_test, y_test = utils.load_reddit_data()
np.random.seed(42)

In [ ]:
# turn the corpus into a list to pass into the vectorizer 
train_corpus = X_train['body'].values.tolist()
test_corpus = X_test['body'].values.tolist()

X_train_tfidf = tfidf_transform(train_corpus, is_train=True)
X_test_tfidf = tfidf_transform(test_corpus, is_train=False)

In [ ]:
X_train_tfidf

In [ ]:
X_test_tfidf

In [7]:
X_train, y_train, X_test, y_test = utils.load_reddit_data()

In [11]:
y_test

,moderated
1054068,1
349140,1
3641685,0
3040687,0
1988232,1
...,...
2681013,0
1680118,1
1725886,1
2330881,0
